### Install Packages

In [ ]:
! pip install transformers simpletransformers

In [ ]:
# %%writefile setup.sh

# git clone https://github.com/NVIDIA/apex
# cd apex
# pip install -v --no-cache-dir ./

In [ ]:
# ! sh setup.sh

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

### Prepare Data

In [ ]:
import numpy as np
import pandas as pd
import json
import os
import re

pd.options.display.max_rows = 300
pd.options.display.max_columns = 100

from IPython.core.debugger import set_trace
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from simpletransformers.classification import ClassificationModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

%load_ext autoreload
%autoreload 2


In [ ]:
df_data = pd.read_csv('../input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv', nrows=10000)

In [ ]:
df_data.rename(columns={'review': 'text', 'sentiment': 'labels'}, inplace=True)

In [ ]:
df_data.head(1)

In [ ]:
# train-validation-test split
df_data_train, df_data_test = train_test_split(df_data, test_size=0.4, random_state=123)
df_data_valid, df_data_test = train_test_split(df_data_test, test_size=0.5, random_state=123)

In [ ]:
print('Shapes:')
print(f'Launch: {df_data_train.shape, df_data_valid.shape, df_data_test.shape}')

### Train Model

In [ ]:
# Create a TransformerModel
model = ClassificationModel('roberta', 'roberta-base', args={'overwrite_output_dir': True})

In [ ]:
# Train the model
model.train_model(train_df=df_data_train, eval_df=df_data_valid)

In [ ]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(df_data_test)

In [ ]:
result['accuracy'] = (result['tp'] + result['tn']) / \
                     (result['tp'] + result['tn'] + result['fp'] + result['fn'])
result['precision'] = result['tp'] / (result['tp'] + result['fp'])
result['recall'] = result['tp'] / (result['tp'] + result['fn'])

result